In [ ]:
#!/usr/bin/env python3

import json
import re
import requests
import copy
import random
import pprint
import matplotlib.pyplot as plt
import pandas as pd
from datetime import date, datetime, timezone, timedelta
import dateutil
from matplotlib_helper import *
from dataclasses import dataclass
from collections import defaultdict

In [ ]:
# Metadata

metadata = {
    'emission_rates': {
        'ylabel': 'gCO2/s',
        'title': 'Instantaneous emission rates'
    },
    'emission_integral': {
        'ylabel': 'gCO2',
        'title': 'Emission integral over its duration'
    },
}

d_timing_labels = {
    "input_transfer_start": "Input transfer",
    # "input_transfer_start": "Start of input transfer",
    # "input_transfer_end": "End of input transfer",
    "compute_start": "Compute",
    # "compute_start": "Start of compute",
    # "compute_end": "End of compute",
    "output_transfer_start": "Output transfer",
    # "output_transfer_start": "Start of output transfer",
    # "output_transfer_end": "End of output transfer",
}

d_events = {
    'input_transfer': {
        'interval_keys': ("input_transfer_start", "input_transfer_end"),
        'label': 'Input transfer',
    },
    'compute': {
        'interval_keys': ("compute_start", "compute_end"),
        'label': 'Compute',
    },
    'output_transfer': {
        'interval_keys': ("output_transfer_start", "output_transfer_end"),
        'label': 'Output transfer',
    },
}

In [ ]:
def get_max_value(data_details: dict, series_name: str):
    max_value = 0
    for region in data_details:
        compute_data = data_details[region][series_name]["compute"]
        transfer_data = data_details[region][series_name]["transfer"]
        max_value = max(max_value, max(compute_data.values(), default=0), max(transfer_data.values(), default=0))
    return max_value

def resample_timeseries(df: pd.DataFrame, interval: str):
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df.set_index("Timestamp", inplace=True)
    df_resampled = df.resample(interval).ffill().reset_index()
    return df_resampled

def create_dataframe_for_plotting(timeseries: dict[str, float], min_start: datetime, max_end: datetime) -> pd.DataFrame:
    """Convert a time series data to a dataframe, while removing out of bound timestamps.
    
        Args:
            timeseries: A dictionary of timestamp strings and values.
            min_start: The minimum cutoff time for the timeseries.
            max_end: The maximum cutoff time for the timeseries.
    """
    timeseries_in_datatime = {datetime.fromisoformat(key): value for key, value in timeseries.items()}
    df = pd.DataFrame(list(timeseries_in_datatime.items()), columns=["Timestamp", "Value"])
    if df.empty:
        return df
    resampled = resample_timeseries(df, "30s")
    mask = (resampled["Timestamp"] >= pd.to_datetime(min_start)) & (resampled["Timestamp"] <= pd.to_datetime(max_end))
    return resampled[mask]

def add_timing(ax, name: str, time: pd.Timestamp, max_value: float, color: str):
    if 'start' in name:
        ax.vlines(time, ymin=0, ymax=max_value, color='gray', alpha=0.5, linestyles="solid" if 'compute' in name else "dashed")
        if 'input' in name:
            ha = 'right'
            rotation = -30
        elif 'output' in name:
            ha = 'left'
            rotation = 30
        else:
            ha = 'center'
            rotation = 0
        ax.text(time, max_value, d_timing_labels[name], color=color, alpha=0.95, ha=ha, va="bottom", rotation=rotation)

In [ ]:
# Plotting

def plot_carbon_api_response(data: dict, show_events=True, show_transfer_breakdown = True, show_compute=True, show_transfer=True):
    fig, axes = plt.subplots(1, 1, figsize=(12, 6))

    d_region_colors = {}
    for region in data['details']:
        d_region_colors[region] = get_next_color()

    # Extract emission integral data
    for (ax, series_name) in zip([axes], ["emission_rates", "emission_integral"]):
        if series_name == 'emission_integral':
            continue
        # # Get max value for y-axis
        # max_y_value = get_max_value(data['details'], series_name)

        for region in data['details']:
            # if region == 'Azure:eastus':
            #     continue
            print(f"Plotting {region} - {series_name}")
            compute_data = data["details"][region][series_name]["compute"]
            transfer_data = data["details"][region][series_name]["transfer"]
            transfer_network_data = data["details"][region][series_name]["transfer.network"]
            transfer_endpoint_data = data["details"][region][series_name]["transfer.endpoint"]
            timings = data["details"][region]['timings'][0] # Assume single occurence per job
            min_start = datetime.fromisoformat(timings['min_start'])
            max_end = datetime.fromisoformat(timings['max_end'])
            carbon_emissions_compute = data['raw-scores'][region]['carbon-emission-from-compute']
            carbon_emissions_transfer = data['raw-scores'][region]['carbon-emission-from-migration']

            # Convert timestamp strings to datetime objects
            compute_df = create_dataframe_for_plotting(compute_data, min_start, max_end)
            transfer_df = create_dataframe_for_plotting(transfer_data, min_start, max_end)
            transfer_network_df = create_dataframe_for_plotting(transfer_network_data, min_start, max_end)
            transfer_endpoint_df = create_dataframe_for_plotting(transfer_endpoint_data, min_start, max_end)

            # Plot timeseries data as step functions
            color = d_region_colors[region]
            if show_compute:
                compute_time = pd.to_timedelta(timings['compute_duration']).floor('s').to_pytimedelta()
                label_compute = f"{region} - Compute ({carbon_emissions_compute:.3f} gCO2 over {compute_time})"
                ax.step(compute_df["Timestamp"], compute_df["Value"], label=label_compute, color=color, linestyle="solid")
            if show_transfer and not transfer_df.empty:
                hop_count = data["details"][region]["route.hop_count"]
                carbon_route_raw_strings = data["details"][region]['route']
                router_hop_isos = '|'.join(filter(lambda x: x is not None, map(lambda x: parse_carbon_route(x, "region"), carbon_route_raw_strings)))
                total_transfer_time = pd.to_timedelta(timings['total_transfer_time']).floor('s').to_pytimedelta()
                if show_events:
                    label_transfer = f"{region} - Transfer ({carbon_emissions_transfer:.3f} gCO2 over {total_transfer_time})"
                else:
                    label_transfer = f"{region} - Transfer ({hop_count} hops: {router_hop_isos})"
                ax.step(transfer_df["Timestamp"], transfer_df["Value"], label=label_transfer, color=color, linestyle="dashed")
                if show_transfer_breakdown:
                    ax.step(transfer_network_df["Timestamp"], transfer_network_df["Value"], label=f"{region} - Transfer (network)", color=color, linestyle="dotted")
                    ax.step(transfer_endpoint_df["Timestamp"], transfer_endpoint_df["Value"], label=f"{region} - Transfer (endpoint)", color=color, linestyle="dashdot")

            # Add events based on the timings
            max_y_value = max(compute_df["Value"].max(), transfer_df["Value"].max())
            for event in d_events if show_events else []:
                df = compute_df if event == 'compute' else transfer_df
                if df.empty:
                    continue
                # Vertical lines and texts
                for name in d_events[event]['interval_keys']:
                    add_timing(ax, name, pd.to_datetime(timings[name]), max_y_value, color=d_region_colors[region])
                # Fill area for events under the curve
                (start_event, end_event) = d_events[event]['interval_keys']
                mask = (df['Timestamp'] >= pd.to_datetime(timings[start_event])) & (df['Timestamp'] <= pd.to_datetime(timings[end_event]))
                alpha = 0.5 if 'compute' in event else 0.25
                ax.fill_between(x=df['Timestamp'], y1=df['Value'], where=mask, color=color, alpha=alpha)

        ax.set_title(metadata[series_name]['title'])
        ax.set_xlabel("Time")
        ax.set_ylabel(metadata[series_name]['ylabel'])
        ax.grid(True)

    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()


In [ ]:
def parse_carbon_route(route: str, output="coordinates"):
    # route = "Router at (39.0127, -77.5342) (emap:US-MIDA-PJM)"
    regex = r"Router at \((?P<lat>-?\d+\.\d+), (?P<lon>-?\d+\.\d+)\) \(emap:(?P<region>.+)\)"
    match = re.match(regex, route)
    if match:
        if output == "coordinates":
            return (float(match.group("lat")), float(match.group("lon")))
        elif output == "region":
            return match.group("region")
        elif output is None:
            return match.groupdict()
        else:
            raise ValueError(f"Invalid output type: {output}")
    else:
        return None

In [ ]:
# Get all regions that we cover

CLOUD_PROVIDERS = ['AWS', 'gcloud']
CARBON_API_HOST = 'http://yak-03.sysnet.ucsd.edu'

cloud_regions_by_provider: dict[str, list[str]] = {}

def get_cloud_regions(cloud_provider: str) -> list[str]:
    """Get a list of regions for a given cloud provider.
    
        Args:
            cloud_provider: The cloud provider name.
    """
    response = requests.get(f"{CARBON_API_HOST}/metadata/cloud-location/locations/{cloud_provider}/")
    assert response.ok, f"Error: API call failed with status code {response.status_code}: {response.text}"
    regions = response.json()
    return list(regions)

for cloud_provider in CLOUD_PROVIDERS:
    cloud_regions = get_cloud_regions(cloud_provider)
    cloud_regions_by_provider[cloud_provider] = cloud_regions
    print(f"{cloud_provider}: {len(cloud_regions)} regions")

# Because they don't have yearly data on the endpoints, we'll ignore these regions
EXCLUDED_REGION_IDS = [
    "AWS:ap-northeast-1",
    "gcloud:asia-northeast1",
    "gcloud:europe-west6",
]

ALL_REGION_IDS = []
for cloud in CLOUD_PROVIDERS:
    for region in cloud_regions_by_provider[cloud]:
        region_id = f"{cloud}:{region}"
        if region_id in EXCLUDED_REGION_IDS:
            continue
        ALL_REGION_IDS.append(region_id)

print('Total regions used:', len(ALL_REGION_IDS))

# pprint.pprint(cloud_regions_by_provider)

In [ ]:
%%script false --no-raise-error

for region_id in ALL_REGION_IDS:
    print(region_id)

## A couple of test payload

In [ ]:
runtime_s = timedelta(seconds=3600).total_seconds()

# See parameter definition in class `Workload` at https://github.com/c3lab-net/energy-data/blob/master/api/models/workload.py
payload = {
    "runtime": runtime_s,
    "schedule": {
        "type": "onetime",
        "start_time": "2022-01-02T00:00:00-00:00",
        "max_delay": 24*3600 - runtime_s
    },
    "dataset": {
        "input_size_gb": 512,
        "output_size_gb": 512,
    },
    # Provide EITHER candidate_providers OR candidate_locations
    "candidate_providers": [
        "AWS",
        "gcloud",
    ],
    # "candidate_locations": [
    #     {
    #         "id": "AWS:us-east-1"
    #     },
    #     {
    #         "id": "AWS:us-west-1"
    #     },
    #     {
    #         "id": "AWS:eu-central-1"
    #     },
    # ],
    "use_prediction": False,
    # emap has the most coverage worldwide
    "carbon_data_source": "emap",
    "watts_per_core": 0,
    "core_count": 20,
    "original_location": "AWS:us-east-1",
    # Note: turn off optimize_carbon if you don't need the timing information, e.g. just raw timeseries carbon data
    "optimize_carbon": False,
    # "use_new_optimization": True,   # defaults value, can ignore
    # Most of the case we consider both compute and network
    "carbon_accounting_mode": "compute-and-network",
    # Accepted values are defined in enum `InterRegionRouteSource` at https://github.com/c3lab-net/energy-data/blob/master/api/models/cloud_location.py
    "inter_region_route_source": "igdb.no-pops",
}

In [ ]:
payload = {'runtime': 141.0, 'schedule': {'type': 'onetime', 'start_time': '2022-04-10T08:18:40+00:00', 'max_delay': 82800.0}, 'dataset': {'input_size_gb': 0.03356373688888889, 'output_size_gb': 0.009619868777777778}, 'candidate_providers': ['AWS'], 'use_prediction': False, 'carbon_data_source': 'emap', 'watts_per_core': 5, 'core_count': 1.52, 'original_location': 'AWS:us-east-1', 'optimize_carbon': True, 'carbon_accounting_mode': 'compute-and-network', 'inter_region_route_source': 'itdk+igdb.no-pops'}

In [ ]:
runtime = timedelta(seconds=3600).total_seconds()
interval = timedelta(days=1).total_seconds()
max_delay = interval - runtime

payload = {'runtime': runtime, 'schedule': {'type': 'onetime', 'start_time': '2022-01-01T00:00:00+00:00', 'max_delay': max_delay}, 'dataset': {'input_size_gb': 1, 'output_size_gb': 1}, 'candidate_providers': ['AWS', 'gcloud'], 'use_prediction': False, 'carbon_data_source': 'emap', 'watts_per_core': 1, 'core_count': 1, 'original_location': 'AWS:us-east-1', 'optimize_carbon': False, 'carbon_accounting_mode': 'compute-and-network', 'inter_region_route_source': 'itdk+igdb.no-pops'}

## Utility functions

In [ ]:
def update_payload(payload: dict, src: str, dsts: list[str] = []):
    payload['original_location'] = src
    if dsts:
        if 'candidate_providers' in payload:
            del payload['candidate_providers']
        payload['candidate_locations'] = [{'id': dst} for dst in dsts]
    else:
        if 'candidate_locations' in payload:
            del payload['candidate_locations']
        payload['candidate_providers'] = CLOUD_PROVIDERS

## Verify we have data for all the ISOs in 2023

In [ ]:
regex_route_iso = re.compile(r'([\w ]+) at \(([-\d.]+), ([-\d.]+)\) \(emap:([\w-]+)\)')

def get_iso_from_hop_str(hop: str) -> str:
    # Example hop: "Router at (39.0469, -77.4903) (emap:US-MIDA-PJM)"
    m = regex_route_iso.match(hop)
    if m:
        return m.group(4)
    else:
        raise ValueError(f'Cannot parse hop: {hop}')

def reverse_region_to_isos_mapping(d: dict[str, list[str]], remove_prefix: str = None) -> dict[str, list[str]]:
    reversed: dict[str, list[str]] = defaultdict(set)
    for region, isos in d.items():
        for iso in isos:
            if remove_prefix and iso.startswith(remove_prefix):
                iso = iso.removeprefix(remove_prefix)
            reversed[iso].add(region)
    return reversed

def parse_warning_no_carbon_data_to_isos(message: str) -> list[str]:
    # Example message: "No carbon data found for iso emap:DZ in time range ..."
    # or "Carbon data not available for iso emap:JP-TK for the entire time range [2023-11-09 00:00:00+00:00, 2023-11-10 00:00:00+00:00]"
    regex = r"(?:No carbon data found for|Carbon data not available for) iso ([\w:-]+) (?:in time range|for the entire time range)"
    match = re.match(regex, message)
    if match:
        return [match.group(1)]
    # or "ISOs with missing carbon data (1): emap:AO, emap:XX."
    regex_multiple_isos = r"ISOs with missing carbon data \(\d+\): ([\w:, -].+)"
    for line in message.splitlines():
        match = re.match(regex_multiple_isos, line, re.MULTILINE)
        if match:
            return match.group(1).split(', ')
    raise ValueError(f'Cannot parse warning message: {message}')

def get_iso_to_affected_region_pairs(data) -> dict[str, list[tuple[str, str]]]:
    """Get the unique ISO and the affected region pairs from the both regions and transit hops.
    
        Args:
            data: The carbon API response.
    """
    # Region route ISOs
    region_to_isos = defaultdict(list)
    for region, d in data['details'].items():
        region_to_isos[region] = [get_iso_from_hop_str(hop) for hop in d['route']]

    # Regions with no carbon data
    if 'warnings' in data:
        for region, message in data['warnings'].items():
            region_to_isos[region] += parse_warning_no_carbon_data_to_isos(message)

    # Region's local ISO
    for region, iso in data['isos'].items():
        region_to_isos[region].append(iso)

    original_location = data['request']['original_location']
    return { iso: [(original_location, region) for region in regions if original_location != region] \
            for iso, regions in reverse_region_to_isos_mapping(region_to_isos, 'emap:').items() }

In [ ]:
# From database, the list of zoneids with full 2023 data
emap_zone_ids_with_full_2023_data = [
    'AT', 'AU-NSW', 'AU-NT', 'AU-QLD', 'AU-SA', 'AU-TAS', 'AU-VIC', 'AU-WA', 'BA', 'BE', 'BG', 'BR', 'BR-CS', 'BR-N', 'BR-NE', 'BR-S', 'CA-ON', 'CA-QC', 'CL-SEN', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DK-BHM', 'DK-DK1', 'DK-DK2', 'EE', 'ES', 'FI', 'FR', 'GB', 'GB-NIR', 'GR', 'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN-EA', 'IN-NE', 'IN-NO', 'IN-SO', 'IN-WE', 'IS', 'IT', 'IT-CNO', 'IT-CSO', 'IT-NO', 'IT-SAR', 'IT-SIC', 'IT-SO', 'JP', 'JP-CB', 'JP-CG', 'JP-HKD', 'JP-HR', 'JP-KN', 'JP-KY', 'JP-ON', 'JP-TH', 'KR', 'LT', 'LU', 'LV', 'NI', 'NL', 'NO', 'NO-NO1', 'NO-NO2', 'NO-NO3', 'NO-NO4', 'NO-NO5', 'NZ', 'PA', 'PE', 'PH', 'PH-LU', 'PH-MI', 'PH-VI', 'PL', 'PT', 'RO', 'RS', 'SE', 'SE-SE1', 'SE-SE2', 'SE-SE3', 'SE-SE4', 'SG', 'SI', 'SK', 'TR', 'TW', 'US', 'US-CAL-BANC', 'US-CAL-CISO', 'US-CAL-IID', 'US-CAL-LDWP', 'US-CAL-TIDC', 'US-CAR-CPLE', 'US-CAR-CPLW', 'US-CAR-DUK', 'US-CAR-SC', 'US-CAR-SCEG', 'US-CAR-YAD', 'US-CENT-SPA', 'US-CENT-SWPP', 'US-FLA-FMPP', 'US-FLA-FPC', 'US-FLA-FPL', 'US-FLA-GVL', 'US-FLA-JEA', 'US-FLA-SEC', 'US-FLA-TAL', 'US-FLA-TEC', 'US-MIDA-PJM', 'US-MIDW-AECI', 'US-MIDW-LGEE', 'US-MIDW-MISO', 'US-NE-ISNE', 'US-NW-AVA', 'US-NW-BPAT', 'US-NW-CHPD', 'US-NW-DOPD', 'US-NW-GCPD', 'US-NW-GRID', 'US-NW-IPCO', 'US-NW-NEVP', 'US-NW-NWMT', 'US-NW-PACE', 'US-NW-PACW', 'US-NW-PGE', 'US-NW-PSCO', 'US-NW-PSEI', 'US-NW-SCL', 'US-NW-TPWR', 'US-NW-WACM', 'US-NW-WAUW', 'US-NY-NYIS', 'US-SE-SOCO', 'US-SW-AZPS', 'US-SW-EPE', 'US-SW-PNM', 'US-SW-SRP', 'US-SW-TEPC', 'US-SW-WALC', 'US-TEN-TVA', 'US-TEX-ERCO', 'UY', 'ZA',
]

In [ ]:
%%script false --no-raise-error
# _debug_

payload = {'runtime': 3600.0, 'schedule': {'type': 'onetime', 'start_time': '2022-12-01T00:00:00+00:00', 'max_delay': 3600*24 - 3600}, 'dataset': {'input_size_gb': 1024, 'output_size_gb': 1024}, 'candidate_providers': ['AWS', 'gcloud'], 'use_prediction': False, 'carbon_data_source': 'emap', 'watts_per_core': 0, 'core_count': 20, 'original_location': 'AWS:us-east-1', 'optimize_carbon': False, 'carbon_accounting_mode': 'compute-and-network', 'inter_region_route_source': 'itdk'}
response = requests.get(CARBON_API_URL, json=payload)

# Check if the API call was successful (status code 200)
assert response.ok, f"Error: API call failed with status code {response.status_code}: {response.text}"
data = response.json(parse_float=lambda s: float('%.6g' % float(s)))
print(json.dumps(data, indent=4))

In [ ]:
PAYLOAD_FULL_DAY_NETWORK_TRANSFER_ONLY = {
    "runtime": 3600,
    "schedule": {
        "type": "onetime",
        "start_time": "2023-01-01T00:00:00+00:00",
        "interval": None,
        "max_delay": 3600 * 24 - 3600
    },
    "dataset": {
        "input_size_gb": 1,
        "output_size_gb": 1
    },
    "original_location": None,
    "candidate_providers": [
        "AWS",
        "gcloud"
    ],
    "candidate_locations": [],
    "carbon_data_source": "emap",
    "use_prediction": False,
    "desired_renewable_ratio": None,
    "optimize_carbon": False,
    "watts_per_core": 0,
    "core_count": 1,
    "use_new_optimization": True,
    "carbon_accounting_mode": "compute-and-network",
    "inter_region_route_source": None
}

### Compare the route estimation heuristics (when certain hops have missing carbon data)

In [ ]:
# ALL_ROUTE_SOURCES = ['itdk', 'igdb.no-pops', 'igdb.with-pops', 'itdk+igdb.no-pops', 'itdk+igdb.with-pops']
ALL_ROUTE_SOURCES = ['itdk+igdb.no-pops', 'itdk+igdb.with-pops']
# All zone coverage starts immediately after 2023/11/21 01:00:00 UTC
START_TIME = datetime(2023, 11, 22, 0, 0, 0, tzinfo=timezone.utc)
DAYS_TO_COVER = 60

CARBON_API_URL='http://yak-03.sysnet.ucsd.edu/carbon-aware-scheduler/'
THREAD_COUNT = 256

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time
import os
import glob
from typing import Any, Optional


def _call_carbon_api_no_throw(payload: dict) -> tuple[Optional[dict], Optional[Any]]:
    """Returns a pair of response JSON and any error message."""
    try:
        response = requests.get(CARBON_API_URL, json=payload, timeout=60)
    except TimeoutError as e:
        return { "request": payload }, f"Error: API call timed out: {e}"
    if response.ok:
        try:
            return response.json(parse_float=lambda s: float('%.6g' % float(s))), None
        except Exception as e:
            return { "request": payload }, f"Error: Failed to parse response JSON: {e}"
    else:
        if response.status_code == 400:
            return response.json(parse_float=lambda s: float('%.6g' % float(s))), None
        else:
            return { "request": payload }, f"Error: API call failed with status code {response.status_code}: {response.text}"
    #end

def set_estimation_heuristic(payload: dict, on: bool,
                             minimum_known_carbon_power_ratio: Optional[float] = None,
                             heuristic: Optional[str] = None,
                             param: Optional[Any] = None):
    if 'network_hop_carbon_estimation_heuristic' in payload:
        del payload['network_hop_carbon_estimation_heuristic']

    if not on:
        payload['network_hop_carbon_estimation_heuristic'] = 'no-estimation'
    else:
        assert minimum_known_carbon_power_ratio is not None
        payload['network_hop_carbon_estimation_minimum_known_carbon_power_ratio'] = \
            minimum_known_carbon_power_ratio

        assert heuristic is not None
        if heuristic == 'route-average':
            payload['network_hop_carbon_estimation_heuristic'] = 'route-average'
            assert param is not None
            payload['network_hop_carbon_estimation_route_average_ratio_threshold'] = \
                param
        elif heuristic == 'nearest-neighbor':
            payload['network_hop_carbon_estimation_heuristic'] = 'nearest-neighbor'
            assert param is not None
            payload['network_hop_carbon_estimation_distance_km_threshold'] = param
        else:
            payload['network_hop_carbon_estimation_heuristic'] = 'world-average'
    #end

def prepare_request_payloads(payload: dict,
                             minimum_known_carbon_power_ratio: Optional[float] = None,
                             heuristic: Optional[str] = None,
                             param: Optional[Any] = None) -> list[dict]:
    payloads: list[dict] = []
    for original_location in ALL_REGION_IDS:
        for route_source in ALL_ROUTE_SOURCES:
            request = copy.deepcopy(payload)
            request['original_location'] = original_location
            request['inter_region_route_source'] = route_source
            # Real case, no estimation
            set_estimation_heuristic(request, False)
            request['only_emap_full_range_isos_for_network_hops'] = False
            payloads.append(request)
            # Estimated case, force certain regions to have no data
            request = copy.deepcopy(request)
            set_estimation_heuristic(request, True,
                                     minimum_known_carbon_power_ratio,
                                     heuristic, param)
            # update_payload(request, original_location, ['gcloud:asia-northeast1'])
            request['only_emap_full_range_isos_for_network_hops'] = True
            payloads.append(request)
    return payloads

def get_network_emission_rates(details, src_region) -> dict[tuple[str, str], pd.Series]:
    emission_rates_by_region_pair: dict[tuple[str, str], pd.Series] = {}
    for dst_region in details:
        # Ignore self-to-self data
        if src_region == dst_region:
            continue
        if dst_region in EXCLUDED_REGION_IDS:
            continue
        region_pair = (src_region, dst_region)
        # temporary issue, need to fix na values in compute in the API
        # ds_compute = pd.Series(details[dst_region]['emission_rates']['compute']).fillna(-1)
        ds_network = pd.Series(details[dst_region]['emission_rates']['transfer.network'])
        emission_rates_by_region_pair[region_pair] = ds_network
    return emission_rates_by_region_pair


In [ ]:
def summarize_error_percents(error_percents: list[float]):
    ds_error_percent = pd.Series(error_percents)
    REGION_PAIR_COUNT = sum(1 for src in ALL_REGION_IDS for dst in ALL_REGION_IDS if src != dst)
    coverage = ds_error_percent.size / REGION_PAIR_COUNT
    mean = ds_error_percent.mean()
    percentile90 = ds_error_percent.quantile(0.9)
    percentile99 = ds_error_percent.quantile(0.99)
    return {
        'coverage': coverage,
        'mean': mean,
        '90th': percentile90,
        '99th': percentile99,
    }

In [ ]:
def get_error_percent_by_region_pair(real_by_region_pair: dict[tuple[str, str], pd.Series],
                                     estimate_by_region_pair: dict[tuple[str, str], pd.Series]) -> dict[tuple[bool, str, str], float]:
    """Get the error percentage for each region pair.
    
        Args:
            network_emission_rates_by_only_full_range_region_pair: A dictionary of region pair to network emission rates.
    """
    region_pairs = set(real_by_region_pair.keys()).intersection(set(estimate_by_region_pair.keys()))
    error_percent_by_region_pair: dict[tuple[str, str], float] = {}
    for region_pair in region_pairs:
        if region_pair not in real_by_region_pair or region_pair not in estimate_by_region_pair:
            continue
        if region_pair[0] == region_pair[1]:
            continue

        ds_real = real_by_region_pair[region_pair]
        ds_estimate = estimate_by_region_pair[region_pair]
        if ds_real.empty or ds_estimate.empty:
            print(f"Empty dataset for {region_pair}: ")
            continue
        # Sometimes they are off by one index
        if not ds_real.index.equals(ds_estimate.index):
            ds_estimate = ds_estimate.reindex(ds_real.index)
        assert ds_real.index.equals(ds_estimate.index), f"Index mismatch for {region_pair}: {ds_real.index} != {ds_estimate.index}"
        ds_error = (ds_real - ds_estimate).abs()
        ds_error_percent = ds_error / ds_real * 100
        if not ds_error_percent[ds_error_percent.isnull()].empty:
            print(f"NaN error percent for {region_pair}")
            print('nan index:', ds_error_percent[ds_error_percent.isnull()].index)
            raise ValueError(f"NaN error percent for {region_pair}")
        error_percent_by_region_pair[region_pair] = ds_error_percent.to_list()
    return error_percent_by_region_pair

In [ ]:
# Graph percentage error on estimation

def flatten(xss):
    return [x for xs in xss for x in xs]

def plot_and_summarize_percent_error(error_percent_by_route_source_region_pair: dict[str, dict[tuple[str, str], float]],
                                     figname: str = None):
    plt.figure(figsize=(12, 6))

    for route_source in ALL_ROUTE_SOURCES:
        print(route_source)
        error_percent_by_region_pair = error_percent_by_route_source_region_pair[route_source]
        l_error_average = flatten(list(error_percent_by_region_pair.values()))
        summary = summarize_error_percents(l_error_average)
        for key, value in summary.items():
            print(f"\t{key}: {value:.2f}")

        # Plot the error difference
        valid_region_pair_count = len(error_percent_by_region_pair)
        plot_cdf_array(l_error_average, label=route_source, include_count=True, override_count=valid_region_pair_count)

    plt.xlabel('Estimation error (%)')
    # curr_lim = plt.xlim()
    # print(curr_lim)
    plt.xlim((-5., 100.))
    plt.grid()
    plt.legend()
    plt.savefig(figname, bbox_inches='tight')


In [ ]:
def get_estimation_heuristic_subfolder_name(min_ratio: str, heuristic: str, param: Optional[Any] = None):
    return f'min_ratio_{min_ratio}.' + heuristic.replace('-', '_') + (f'.{param}' if param and heuristic == 'nearest-neighbor' else '')

In [ ]:
base_payload = copy.deepcopy(PAYLOAD_FULL_DAY_NETWORK_TRANSFER_ONLY)

# base_payload['carbon_accounting_mode'] = 'compute-only'
base_payload['schedule']['start_time'] = START_TIME.isoformat()
base_payload['schedule']['max_delay'] = timedelta(days=DAYS_TO_COVER).total_seconds() - base_payload['runtime']
# print(base_payload['schedule']['max_delay'])

HEURISTIC_PARAM_TUPLES = [
    (0.75, 'route-average', 0.5),
    # (0.5, 'nearest-neighbor', 500),
    # (0.5, 'nearest-neighbor', 1500),
    (0.75, 'nearest-neighbor', 50000),
    (0.75, 'world-average', None)
]

# def get_response_file_path(request: dict) -> str:
#     original_location = request['original_location']
#     route_source = request['inter_region_route_source']
#     is_estimated = bool(request['only_emap_full_range_isos_for_network_hops'])
#     filename = f'response.{original_location}.{is_estimated}.{route_source}.json'
#     return f"jsons/{payload['inter_region_route_source']}.{payload['original_location']}.json"

for min_ratio, heuristic, param in HEURISTIC_PARAM_TUPLES:
    print('Heuristic:', heuristic, 'Param:', param)
    subfolder = get_estimation_heuristic_subfolder_name(min_ratio, heuristic, param)
    os.makedirs(f'jsons/{subfolder}', exist_ok=True)
    payloads = prepare_request_payloads(base_payload, min_ratio, heuristic, param)
    print(f'Added {len(payloads)} requests to the queue')

    perf_start_time = time.time()
    has_errors = False
    response_count = 0
    # random.shuffle(payloads)

    real_network_emission_rates_by_route_source_region_pair: dict[str, dict[tuple[str, str], pd.Series]] = {}
    estimated_network_emission_rates_by_route_source_region_pair: dict[str, dict[tuple[str, str], pd.Series]] = {}
    for route_source in ALL_ROUTE_SOURCES:
        real_network_emission_rates_by_route_source_region_pair[route_source] = {}
        estimated_network_emission_rates_by_route_source_region_pair[route_source] = {}
    with ThreadPoolExecutor(THREAD_COUNT) as pool:
        for response, error in pool.map(_call_carbon_api_no_throw, payloads):
            original_location = response['request']['original_location']
            route_source = response['request']['inter_region_route_source']
            is_estimated = bool(response['request']['only_emap_full_range_isos_for_network_hops'])
            filename = f'response.{original_location}.{is_estimated}.{route_source}.json'
            filepath = f"jsons/{subfolder}/{filename}"
            json.dump(response, open(filepath, "w"), indent=4)
            elapsed = time.time() - perf_start_time
            response_count += 1
            print(f'{response_count} {elapsed:.2f} {original_location} - {route_source}, {is_estimated}: {"failure" if error else "success"}', file=sys.stderr)
            if error:
                response["error"] = error
                has_errors = True
                break
            network_emission_rates = get_network_emission_rates(response['details'], original_location)
            for region_pair, series in network_emission_rates.items():
                if is_estimated:
                    assert region_pair not in estimated_network_emission_rates_by_route_source_region_pair[route_source]
                    estimated_network_emission_rates_by_route_source_region_pair[route_source][region_pair] = series
                else:
                    assert region_pair not in real_network_emission_rates_by_route_source_region_pair[route_source]
                    real_network_emission_rates_by_route_source_region_pair[route_source][region_pair] = series
    # End of ThreadPoolExecutor
    if has_errors:
        print('WARNING!!! There are errors in the responses. Please check the JSON files.')
    else:
        print('All done.')

    # Process results
    error_percent_by_route_source_region_pair: dict[str, dict[tuple[str, str], float]] = {}
    for route_source in ALL_ROUTE_SOURCES:
        error_percent_by_route_source_region_pair[route_source] = get_error_percent_by_region_pair(
            real_network_emission_rates_by_route_source_region_pair[route_source],
            estimated_network_emission_rates_by_route_source_region_pair[route_source])
    plot_and_summarize_percent_error(error_percent_by_route_source_region_pair,
                                     figname=f'plots/percent_error.{subfolder}.png')

In [ ]:
# Analysis on route carbon estimation:

import math


def get_transfer_power_with_iso_ratios(response) -> dict[tuple[str, str], float]:
    """Get the transfer power with ISO ratio on a per-region-pair basis.
    
        Args:
            data: The carbon API response.
    """
    ratios_by_region_pair: dict[tuple[str, str], float] = {}
    src_region = response['request']['original_location']
    if 'error' in response:
        return {}
    for dst_region, d in response['details'].items():
        if src_region == dst_region:
            continue
        if src_region in EXCLUDED_REGION_IDS or dst_region in EXCLUDED_REGION_IDS:
            continue
        ratio = d.get('transfer.network.power_ratio_with_carbon_data', math.nan)
        ratios_by_region_pair[(src_region, dst_region)] = ratio
    return ratios_by_region_pair

plt.figure(figsize=(12, 6))
region_pairs_without_this_ratio = set()
for route_source in ALL_ROUTE_SOURCES:
    all_transfer_power_with_iso_ratios = {}
    for src_region in ALL_REGION_IDS:
        subfolder = get_estimation_heuristic_subfolder_name(0, 'world-average')
        data = json.load(open(f"jsons/{subfolder}/response-{src_region}.True.{route_source}.json"))
        transfer_power_with_iso_ratios = get_transfer_power_with_iso_ratios(data)
        # for t in transfer_power_with_iso_ratios.keys():
        #     if t[0] == t[1]:
        #         print(t)
        regions_without_this_ratio = set()
        if 'warnings' in data:
            regions_without_this_ratio.update(data['warnings'].keys())
        if 'error' in data:
            regions_without_this_ratio.update(data['details'].keys())
        regions_without_this_ratio.add(src_region)
        expected_regions = set(ALL_REGION_IDS).difference(regions_without_this_ratio)
        actual_regions = set(t[1] for t in transfer_power_with_iso_ratios.keys())

        # print(original_location, len(actual_regions),
        #       len(data['warnings']) if 'warnings' in data else 0,
        #       len(data['details']) if 'error' in data else 0)

        assert actual_regions == expected_regions, f"Missing region pair for {src_region} - {route_source}. Expected: {expected_regions}, Actual: {actual_regions}, difference: {expected_regions.difference(actual_regions)}"

        region_pairs_without_this_ratio.update((src_region, dst) for dst in regions_without_this_ratio \
                                               if src_region != dst)
        all_transfer_power_with_iso_ratios.update(transfer_power_with_iso_ratios)
    plot_cdf_array(list(all_transfer_power_with_iso_ratios.values()), label=route_source, include_count=True)
    print(route_source, len(all_transfer_power_with_iso_ratios))
plt.legend()
plt.grid()
plt.xlabel("Ratio of network power with carbon data")
# plt.title(f"Ratio of transfer power with carbon data")
plt.savefig('network_power_with_carbon_data.ratio.png')

# unique_regions = set(p[0] for p in region_pairs_without_this_ratio).union(
#     set(p[1] for p in region_pairs_without_this_ratio))
# print('unique regions:', len(unique_regions))
# for region in unique_regions:
#     print(region)
# print()

# print('region pairs without ratio:', len(region_pairs_without_this_ratio))
# for region_pair in region_pairs_without_this_ratio:
#     print(region_pair)


In [ ]:
for route_source in ALL_ROUTE_SOURCES[:1]:
    all_affected_region_pairs = set()
    all_isos_to_region_pairs = defaultdict(set)
    for src_region in ALL_REGION_IDS:
        subfolder = get_estimation_heuristic_subfolder_name(0, 'world-average')
        data = json.load(open(f"jsons/{subfolder}/response-{src_region}.True.{route_source}.json"))
        isos_to_region_pairs = get_iso_to_affected_region_pairs(data)
        for iso, region_pairs in isos_to_region_pairs.items():
            all_isos_to_region_pairs[iso].update(region_pairs)
    isos = list(all_isos_to_region_pairs.keys())
    print(f"Found {len(isos)} unique ISOs under {route_source}: {isos}")

    for iso in all_isos_to_region_pairs:
        if iso in emap_zone_ids_with_full_2023_data:
            continue
        # print(f"Warning: {iso} is not in the list of EMAP zone IDs with full 2023 data. "
        #         f"Affected regions: {all_isos_to_region_pairs[iso]}")
        all_affected_region_pairs.update(all_isos_to_region_pairs[iso])
    print(f"Found {len(all_affected_region_pairs)} unique affected region pairs under {route_source}:")
    print(all_affected_region_pairs)

In [ ]:
# Profile individual workloads

# Load payload from above
assert payload, "Error: payload is not defined"

# Restrict to two locations
if 'candidate_providers' in payload:
    del payload['candidate_providers']
payload['candidate_locations'] = [
    {
        "id": "AWS:us-east-1"
    },
    {
        "id": "AWS:us-west-1"
    },
]

@dataclass
class Workload:
    name: str
    core_count: int
    runtime_s: float
    input_gb: float
    output_gb: float

"""csv
name,core_count,runtime_s,input_gb,output_gb
compression (gzip),13,2.1,1.2,0.19
compression (bzip),31,3.2,1.2,0.142
code compilation (linux v5.16),40,391,1.2,0.195
video resize (4k->1080p),4,261,4.6,0.251
video effect (4k, h.264),8,307,1.2,0.122
video effect (4k, h.265),8,307,0.15,0.117
ML inference (resnet50, 1k images),1,246,0.154,0
"""

profiled_workloads = [
    Workload("compression (gzip)", 13, 2.1, 1.2, 0.19),
    Workload("compression (bzip)", 31, 3.2, 1.2, 0.142),
    Workload("code compilation (linux v5.16)", 40, 391, 1.2, 0.195),
    Workload("video resize (4k->1080p)", 4, 261, 4.6, 0.251),
    Workload("video effect (4k, h.264)", 8, 307, 1.2, 0.122),
    Workload("video effect (4k, h.265)", 8, 307, 0.15, 0.117),
    Workload("ML inference (resnet50, 1k images)", 1, 246, 0.154, 0),
]

CARBON_API_URL = 'http://yak-03.sysnet.ucsd.edu/carbon-aware-scheduler/'

for workload in profiled_workloads:
    workload.runtime_s *= 10
    workload.input_gb *= 10
    workload.output_gb *= 10
    payload['runtime'] = workload.runtime_s
    payload['schedule']['max_delay'] = timedelta(days=1).total_seconds() - workload.runtime_s
    payload['dataset']['input_size_gb'] = workload.input_gb
    payload['dataset']['output_size_gb'] = workload.output_gb
    payload['core_count'] = workload.core_count
    print(f'{workload.name} - {workload.core_count}c * {workload.runtime_s}s', f'{workload.input_gb}G/{workload.output_gb}G')
    print('Core-hours/GB:', workload.core_count * workload.runtime_s / timedelta(hours=1).total_seconds() / (workload.input_gb + workload.output_gb))

    response = requests.get(CARBON_API_URL, json=payload)
    assert response.ok, f"Error: API call failed with status code {response.status_code}: {response.text}"
    data = response.json(parse_float=lambda s: float('%.6g' % float(s)))
    # print(json.dumps(data, indent=4))
    plot_carbon_api_response(data)



In [ ]:
# Calculate average/min/max transfer carbon cost across all region pairs

# TODO: check this code from copilot
def get_carbon_cost_for_region_pair(source_region: str, destination_region: str, payload: dict) -> dict[time, float]:
    """Calculate the carbon cost for a region pair.
    
        Args:
            source_region: The source region.
            destination_region: The destination region.
            payload: The payload to be sent to the API.
    """
    payload['candidate_locations'] = [
        {
            "id": source_region
        },
        {
            "id": destination_region
        },
    ]
    response = requests.get(CARBON_API_URL, json=payload)
    assert response.ok, f"Error: API call failed with status code {response.status_code}: {response.text}"
    data = response.json(parse_float=lambda s: float('%.6g' % float(s)))
    carbon_emissions_transfer = data['raw-scores'][destination_region]['carbon-emission-from-migration']
    return carbon_emissions_transfer


In [ ]:
# Ad-hoc: verify that new optimization is no worse than the old one, by sweeping over all parameters

payload = {
    "runtime": 5400,
    "schedule": {
        "type": "onetime",
        "start_time": "2023-05-24T22:00:00-05:00",
        "max_delay": 36000
    },
    "dataset": {
        "input_size_gb": 20,
        "output_size_gb": 12
    },
    "candidate_locations": [
        {
            "id": "AWS:us-east-1"
        },
        {
            "id": "AWS:us-west-1"
        },
        # {
        #     "id": "AWS:us-west-1"
        # },
        # {
        #     "id": "AWS:us-west-1"
        # },
        # {
        #     "id": "AWS:us-west-1"
        # },
        # {
        #     "id": "AWS:us-west-1"
        # },
        # {
        #     "id": "AWS:us-west-1"
        # },
    ],
    "use_prediction": False,
    "carbon_data_source": "emap",
    "watts_per_core": 5,
    "core_count": 40,
    "original_location": "AWS:us-east-1",
    "carbon_accounting_mode": "compute-and-network",
    "optimize_carbon": True,
}


def compare_results(payload1: dict, payload2: dict):
    response1 = requests.get(CARBON_API_URL, json=payload1)
    response2 = requests.get(CARBON_API_URL, json=payload2)
    if response1.ok != response2.ok:
        print("Inconsistent response status")
        print(payload)
        print(response1)
        print(response2)
        return
    elif response1.ok == False:
        print("Both responses are not ok")
        print(payload)
        print(response1.status_code)
        print(response1.json())
        return
    parse_float=lambda s: float('%.9g' % float(s))
    json1 = response1.json(parse_float=parse_float)
    json2 = response2.json(parse_float=parse_float)
    del json1['request']
    del json2['request']
    if json1 != json2:
        should_report = False
        for region in json2['raw-scores']:
            if json2['raw-scores'][region]['carbon-emission'] > json1['raw-scores'][region]['carbon-emission']:
                print(f"Worse result for {region}")
                should_report = True
        if should_report:
            print("Inconsistent response content")
            print(payload)
            print(json1)
            print(json2)
            return
    else:
        print("Same result")
    print(json1)
    print(json2)
    print('old:', end='')
    for region in json1['raw-scores']:
        print('\t', region, json1['raw-scores'][region]['carbon-emission'], end='')
    print()
    print('new:', end='')
    for region in json2['raw-scores']:
        print('\t', region, json2['raw-scores'][region]['carbon-emission'], end='')
    print()

pp = pprint.PrettyPrinter(indent=4)

CARBON_API_URL='http://yak-03.sysnet.ucsd.edu/carbon-aware-scheduler/'

# # Fuzzing errorneous input
# payloads = [
#     {'runtime': 14802, 'schedule': {'type': 'onetime', 'start_time': '2022-08-18T12:18:57+00:00', 'max_delay': 36000}, 'dataset': {'input_size_gb': 20, 'output_size_gb': 69}, 'candidate_locations': [{'id': 'AWS:us-east-1'}, {'id': 'AWS:us-west-1'}], 'use_prediction': False, 'carbon_data_source': 'emap', 'watts_per_core': 5, 'core_count': 5, 'original_location': 'AWS:us-east-1', 'carbon_accounting_mode': 'compute-and-network', 'optimize_carbon': True, 'use_new_optimization': False},
# ]

# for payload in payloads:
#     payload_new_optimization = copy.deepcopy(payload)
#     payload_new_optimization['use_new_optimization'] = True
#     compare_results(payload, payload_new_optimization)

# Make the API call
for i in range(365):
    for runtime_seconds in [random.randint(1, 3600 * 24) for _ in range(1)]:
        START_TIME = datetime(2022, 1, 1, 0, 0, 0, tzinfo=timezone.utc) + timedelta(days=i)
        START_TIME += timedelta(seconds=random.randint(0, 3600 * 24))
        data_input_size_gb = random.randint(1, 100)
        data_output_size_gb = random.randint(1, 100)
        core_count = random.randint(1, 100)

        payload['schedule']['start_time'] = START_TIME.isoformat()
        payload['runtime'] = runtime_seconds
        payload['dataset']['input_size_gb'] = data_input_size_gb
        payload['dataset']['output_size_gb'] = data_output_size_gb
        payload['core_count'] = core_count
        payload['use_new_optimization'] = False
        print(START_TIME, f'{core_count}c * {runtime_seconds}s', f'{data_input_size_gb}G/{data_output_size_gb}G')

        payload_new_optimization = copy.deepcopy(payload)
        payload_new_optimization['use_new_optimization'] = True
        compare_results(payload, payload_new_optimization)
